In [1]:
import os
import pandas as pd

In [ ]:
#Example: load book_train for stock_id=0
df_book0 = pd.read_parquet("trade_train.parquet/stock_id=0")
print(df_book0.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123443 entries, 0 to 123442
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   time_id            123443 non-null  int16  
 1   seconds_in_bucket  123443 non-null  int16  
 2   price              123443 non-null  float32
 3   size               123443 non-null  int32  
 4   order_count        123443 non-null  int16  
dtypes: float32(1), int16(3), int32(1)
memory usage: 1.6 MB
None


In [11]:
print(df_book0.head())


   time_id  seconds_in_bucket     price  size  order_count
0        5                 21  1.002301   326           12
1        5                 46  1.002778   128            4
2        5                 50  1.002818    55            1
3        5                 57  1.003155   121            5
4        5                 68  1.003646     4            1


In [16]:
import pandas as pd
import numpy as np
import plotly.express as px
train = pd.read_csv('/Users/alex/Documents/GitHub/TradingBot/TradingBot/train.csv')
train.head()



,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [19]:
book_example = pd.read_parquet('/Users/alex/Documents/GitHub/TradingBot/TradingBot/book_train.parquet/stock_id=0')
trade_example =  pd.read_parquet('/Users/alex/Documents/GitHub/TradingBot/TradingBot/trade_train.parquet/stock_id=0')
stock_id = '0'
book_example = book_example[book_example['time_id']==5]
book_example.loc[:,'stock_id'] = stock_id
trade_example = trade_example[trade_example['time_id']==5]
trade_example.loc[:,'stock_id'] = stock_id


In [20]:
book_example.head()


,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0


In [21]:
trade_example.head()


,time_id,seconds_in_bucket,price,size,order_count,stock_id
0,5,21,1.002301,326,12,0
1,5,46,1.002778,128,4,0
2,5,50,1.002818,55,1,0
3,5,57,1.003155,121,5,0
4,5,68,1.003646,4,1,0


In [22]:
book_example['wap'] = (book_example['bid_price1'] * book_example['ask_size1'] +
                                book_example['ask_price1'] * book_example['bid_size1']) / (
                                       book_example['bid_size1']+ book_example['ask_size1'])

In [23]:
fig = px.line(book_example, x="seconds_in_bucket", y="wap", title='WAP of stock_id_0, time_id_5')
fig.show()

In [24]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

In [25]:
book_example.loc[:,'log_return'] = log_return(book_example['wap'])
book_example = book_example[~book_example['log_return'].isnull()]


In [26]:
fig = px.line(book_example, x="seconds_in_bucket", y="log_return", title='Log return of stock_id_0, time_id_5')
fig.show()

In [27]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
realized_vol = realized_volatility(book_example['log_return'])
print(f'Realized volatility for stock_id 0 on time_id 5 is {realized_vol}')

Realized volatility for stock_id 0 on time_id 5 is 0.004499364172786558


In [28]:
import os
from sklearn.metrics import r2_score
import glob
list_order_book_file_train = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/*')

In [29]:
def realized_volatility_per_time_id(file_path, prediction_column_name):
    df_book_data = pd.read_parquet(file_path)
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data[
                                  'ask_size1'])
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    return df_realized_vol_per_stock[['row_id',prediction_column_name]]

In [30]:
def past_realized_volatility_per_stock(list_file,prediction_column_name):
    df_past_realized = pd.DataFrame()
    for file in list_file:
        df_past_realized = pd.concat([df_past_realized,
                                     realized_volatility_per_time_id(file,prediction_column_name)])
    return df_past_realized
df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
                                                           prediction_column_name='pred')


In [33]:
import pandas as pd

# ---------- utilities ----------
def ensure_keys(df: pd.DataFrame) -> pd.DataFrame:
    """Make sure df has stock_id and time_id columns.
       Accepts either existing columns or a row_id (column or index) like '3-57' and splits it.
    """
    # If both keys already exist, done.
    if {'stock_id', 'time_id'}.issubset(df.columns):
        return df

    # If row_id is the index, reset it to a column first.
    if df.index.name == 'row_id' or ('row_id' in getattr(df.index, 'names', [])):
        df = df.reset_index()

    # If row_id is a column, split "stock-time" into two ints.
    if 'row_id' in df.columns:
        parts = df['row_id'].astype(str).str.split('-', n=1, expand=True)
        if parts.shape[1] == 2:
            df = df.assign(
                stock_id=parts[0].astype(int),
                time_id=parts[1].astype(int)
            )
            return df

    raise ValueError(
        "Need join keys: provide ['stock_id','time_id'] or a 'row_id' (column/index) like '3-57'."
    )

def find_pred_col(df: pd.DataFrame) -> str:
    """Find the prediction/target column name to rename to 'pred' for the merge."""
    for c in ['pred', 'prediction', 'preds', 'y_pred', 'target',
              'past_realized_volatility', 'realized_volatility', 'y', 'label']:
        if c in df.columns:
            return c
    raise KeyError("Could not find a prediction/target column in df_past_realized_train.")

# ---------- prepare keys ----------
train = ensure_keys(train)
df_past_realized_train = ensure_keys(df_past_realized_train)

# ---------- choose prediction column on the right ----------
right_pred_col = find_pred_col(df_past_realized_train)

# ---------- merge on (stock_id, time_id) ----------
df_joined = (
    train[['stock_id', 'time_id', 'target']]
      .merge(
          df_past_realized_train[['stock_id', 'time_id', right_pred_col']]
              .rename(columns={right_pred_col: 'pred'}),
          on=['stock_id', 'time_id'],
          how='left'
      )
)

# (optional) add row_id for convenience
df_joined['row_id'] = (
    df_joined['stock_id'].astype(str) + '-' + df_joined['time_id'].astype(str)
)

print(df_joined.head())


SyntaxError: unterminated string literal (detected at line 49) (3965437807.py, line 49)